In [13]:
# Importing useful dependencies
import boto3
import numpy as np
from PIL import Image
from io import BytesIO
import torch
import random
from torchvision import transforms
import torchvision.transforms.functional as TF
# Set a seed for reproducibility
SEED = 10721
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [2]:
# Setup S3 client for MinIO (MinIO implements Amazon S3 API)
s3 = boto3.client(
    "s3",
    endpoint_url="http://127.0.0.1:9000", # MinIO API endpoint
    aws_access_key_id="minioadmin", # User name
    aws_secret_access_key="minioadmin", # Password
)

In [3]:
# We create a new Bucket in Min-IO to store our augmented data

# List existing buckets
buckets = [b["Name"] for b in s3.list_buckets()["Buckets"]]

# Function that given a name, creates a bucket
def createBucket(name, list_buckets):
    if name in list_buckets:
        print(f"Bucket '{name}' already exists!")
    else:
        s3.create_bucket(Bucket=name)
        print(f"Created bucket: {name}")

# Create a bucket named landing_zone
createBucket("training-data-construction-zone", buckets)
# Sub-bucket: Baseline Training Data
s3.put_object(Bucket="training-data-construction-zone", Key="images/")

Bucket 'training-data-construction-zone' already exists!


{'ResponseMetadata': {'RequestId': '187A105602831645',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-checksum-crc32': 'AAAAAA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '187A105602831645',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '2109',
   'x-ratelimit-remaining': '2109',
   'x-xss-protection': '1; mode=block',
   'date': 'Fri, 21 Nov 2025 15:49:25 GMT'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ChecksumCRC32': 'AAAAAA==',
 'ChecksumType': 'FULL_OBJECT'}

In [4]:
# We can use this function to retrieve an image from our bucket in PIL Image format
def get_image(bucket, key):
    resp = s3.get_object(Bucket=bucket, Key=key)
    body = resp["Body"].read()
    img = Image.open(BytesIO(body))
    return img
    

In [30]:
# resize helper used after cropping
resize_to_512 = transforms.Resize((512, 512))  # resize image to 512x512 pixels
def generate_and_save_augmented(
    transformer,            # any callable that takes a PIL image and returns a PIL image
    suffix: str,            # suffix to distinguish different augmentations, e.g. "hflip", "vflip", "crop"
    image: Image.Image,     # original PIL image to be augmented
    dest_bucket: str,       # name of the destination S3 bucket
    dest_prefix: str,       # prefix (folder path) inside the bucket, e.g. "augmented/"
    new_key_infix: str,     # main part of the file name (usually derived from the original key)
):
    # apply the augmentation to the input image
    if suffix == "crop":
        # For random crops we sample a region per image, so we capture the crop parameters to know exactly what was applied.
        # sample crop params
        i, j, h, w = transformer.get_params(image, output_size=transformer.size)
        # apply crop manually
        aug_image = TF.crop(image, i, j, h, w)
        # resize back to 512x512
        aug_image = resize_to_512(aug_image)
        print(
            f"[{new_key_infix}]: [crop] cropped region - top={i}, left={j}, "
            f"height={h}, width={w}"
        )
    else:
        # For flips we always apply the transform (p=1.0), so the behavior is deterministic for each image.
        aug_image = transformer(image)
        print(f"[{new_key_infix}]: [{suffix}] augmentation applied by transformer.")

    # build the new S3 object key, e.g. "images/image000123_hflip.png"
    augment_key = f"{dest_prefix}{new_key_infix}_{suffix}.png"

    # serialize PIL image to bytes (in-memory buffer) as PNG
    buffer = BytesIO()
    aug_image.save(buffer, format="PNG")
    buffer.seek(0)

    # upload the augmented image bytes to S3
    s3.upload_fileobj(
        buffer,
        Bucket=dest_bucket,
        Key=augment_key,
        ExtraArgs={"ContentType": "image/png"}
    )

In [17]:
def image_augmentation(src_bucket, dest_bucket, dest_prefix="images/"):
    id_counter = 0
    
    # define individual image augmentations
    hflip_transform = transforms.RandomHorizontalFlip(p=1.0)  # always flip image horizontally (left-right)
    vflip_transform = transforms.RandomVerticalFlip(p=1.0)    # always flip image vertically (top-bottom)

    # random crop 
    crop_transform = transforms.RandomCrop(400) # randomly crop a 400x400 patch from the original image
    paginator = s3.get_paginator("list_objects_v2") # It returns objects in pages and not all at once.
    for page in paginator.paginate(Bucket=src_bucket, Prefix="baseline-training-data/"):

        for obj in page.get("Contents", []):
            key = obj["Key"]

            if "image" in key:
                # get image
                image = get_image(src_bucket,key)
                # new key of image
                new_key_infix = key.split("/")[1].split(".")[0]
                new_key = dest_prefix + new_key_infix + ".png"
                copy_source_text = {"Bucket": src_bucket, "Key": key}
                s3.copy_object(Bucket=dest_bucket, Key=new_key, CopySource=copy_source_text)

                #generate image flip horizontally
                generate_and_save_augmented(hflip_transform,"x_rotate",image,dest_bucket,dest_prefix,new_key_infix)
                #generate image flip vertically
                generate_and_save_augmented(vflip_transform,"y_rotate",image,dest_bucket,dest_prefix,new_key_infix)
                #generate image cropped
                generate_and_save_augmented(crop_transform,"crop",image,dest_bucket,dest_prefix,new_key_infix)
                


In [31]:
image_augmentation(src_bucket = "training-data-construction-zone", dest_bucket = "training-data-construction-zone")

[image_000001]: [x_rotate] augmentation applied by transformer.
[image_000001]: [y_rotate] augmentation applied by transformer.
[image_000001]: [crop] cropped region - top=69, left=69, height=400, width=400
[image_000002]: [x_rotate] augmentation applied by transformer.
[image_000002]: [y_rotate] augmentation applied by transformer.
[image_000002]: [crop] cropped region - top=13, left=111, height=400, width=400
[image_000003]: [x_rotate] augmentation applied by transformer.
[image_000003]: [y_rotate] augmentation applied by transformer.
[image_000003]: [crop] cropped region - top=93, left=39, height=400, width=400
[image_000004]: [x_rotate] augmentation applied by transformer.
[image_000004]: [y_rotate] augmentation applied by transformer.
[image_000004]: [crop] cropped region - top=24, left=81, height=400, width=400
[image_000005]: [x_rotate] augmentation applied by transformer.
[image_000005]: [y_rotate] augmentation applied by transformer.
[image_000005]: [crop] cropped region - top